## Configuration

Modify these parameters to customize the experiments.

In [ ]:
# Fuzzy Classifier Parameters
CONFIG = {
    'nRules': 40,                    # Number of fuzzy rules
    'nAnts': 3,                      # Number of antecedents per rule
    'n_linguistic_variables': 3,     # Number of linguistic variables per feature
    
    # Training Parameters
    'n_gen': 30,                     # Number of generations
    'pop_size': 60,                  # Population size
    'random_state': 42,              # Random seed for reproducibility
    
    # Genetic Algorithm Parameters
    'sbx_eta': 20.0,                 # SBX crossover distribution index
    'mutation_eta': 20.0,            # Polynomial mutation distribution index
    'var_prob': 0.3,                 # Crossover probability
    
    # Dataset Parameters
    'test_size': 0.3,                # Proportion of test set
    'large_dataset_samples': 100000,   # Samples for large dataset test
    'large_dataset_features': 8,     # Features for large dataset test
}

print("📝 Configuration loaded:")
print(f"   Fuzzy Rules: {CONFIG['nRules']} rules with {CONFIG['nAnts']} antecedents")
print(f"   Training: {CONFIG['n_gen']} generations, population {CONFIG['pop_size']}")

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import time
from sklearn.datasets import load_iris, make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

try:
    import matplotlib.pyplot as plt
    MATPLOTLIB_AVAILABLE = True
except ImportError:
    MATPLOTLIB_AVAILABLE = False
    print("⚠️  matplotlib not available. Visualizations will be skipped.")

from ex_fuzzy import evolutionary_fit as GA_RB
from ex_fuzzy import evolutionary_backends
import ex_fuzzy

print("✅ Libraries imported successfully")

## Check Available Backends and Hardware

In [ ]:
print("Checking available backends and hardware...\n")

# Check available backends
print("Available backends:")
available = evolutionary_backends.list_available_backends()
for backend in available:
    print(f"  ✓ {backend}")

# Check if GPU is available for EvoX
if 'evox' in available:
    try:
        import torch
        if torch.cuda.is_available():
            print(f"\n🎉 GPU detected! EvoX will use GPU acceleration.")
            print(f"   Device: {torch.cuda.get_device_name(0)}")
        else:
            print("\n⚠️  No GPU detected. EvoX will run on CPU.")
    except:
        print("⚠️  Could not check PyTorch GPU availability")
else:
    print("\n⚠️  EvoX not available. Only pymoo backend will be tested.")
    print("   To enable EvoX: pip install evox torch")

## Load and Prepare Dataset

We'll use the Iris dataset for this demo.

In [ ]:
# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
class_names = iris.target_names

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=CONFIG['test_size'], random_state=CONFIG['random_state'], stratify=y
)

print("Dataset Information:")
print(f"  Training samples: {len(X_train)}")
print(f"  Test samples: {len(X_test)}")
print(f"  Features: {len(feature_names)}")
print(f"  Classes: {list(class_names)}")
print(f"\nFeatures: {list(feature_names)}")

## Test PyMoo Backend (Traditional CPU)

First, let's test the traditional PyMoo backend for reference.

In [ ]:
print("Training with PyMoo backend...\n")

# Construct fuzzy partitions
fv_partitions = ex_fuzzy.utils.construct_partitions(
    X_train, n_partitions=CONFIG['n_linguistic_variables']
)

# Create classifier with pymoo backend
clf_pymoo = GA_RB.BaseFuzzyRulesClassifier(
    nRules=CONFIG['nRules'],
    nAnts=CONFIG['nAnts'],
    n_linguistic_variables=CONFIG['n_linguistic_variables'],
    verbose=True,
    backend='pymoo',  # Explicitly specify pymoo
    linguistic_variables=fv_partitions
)

# Train
start_time = time.time()
clf_pymoo.fit(
    X_train,
    y_train,
    n_gen=CONFIG['n_gen'],
    pop_size=CONFIG['pop_size'],
    random_state=CONFIG['random_state']
)
pymoo_time = time.time() - start_time

# Evaluate
y_pred_pymoo = clf_pymoo.predict(X_test)
pymoo_accuracy = accuracy_score(y_test, y_pred_pymoo)
pymoo_n_rules = len(clf_pymoo.rule_base.get_rules())

print(f"\n{'=' * 70}")
print("PYMOO RESULTS:")
print(f"{'=' * 70}")
print(f"  Training time: {pymoo_time:.2f} seconds")
print(f"  Test accuracy: {pymoo_accuracy:.4f}")
print(f"  Number of rules: {pymoo_n_rules}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred_pymoo, target_names=class_names))

### Display Learned Rules (PyMoo)

In [ ]:
print("Learned Fuzzy Rules (PyMoo):")
print("=" * 70)
rule_str = clf_pymoo.rule_base.print_rules(return_rules=True)
print(rule_str)

## Test EvoX Backend (GPU-Accelerated)

Now let's test the new EvoX backend with GPU acceleration.

In [ ]:
if 'evox' in available:
    print("Training with EvoX backend...\n")
    
    # Construct fuzzy partitions (reuse or create new)
    fv_partitions = ex_fuzzy.utils.construct_partitions(
        X_train, n_partitions=CONFIG['n_linguistic_variables']
    )
    
    # Create classifier with evox backend
    clf_evox = GA_RB.BaseFuzzyRulesClassifier(
        nRules=CONFIG['nRules'],
        nAnts=CONFIG['nAnts'],
        n_linguistic_variables=CONFIG['n_linguistic_variables'],
        verbose=True,
        backend='evox',  # Use EvoX backend
        linguistic_variables=fv_partitions
    )
    
    # Train
    start_time = time.time()
    clf_evox.fit(
        X_train,
        y_train,
        n_gen=CONFIG['n_gen'],
        pop_size=CONFIG['pop_size'],
        random_state=CONFIG['random_state'],
        sbx_eta=CONFIG['sbx_eta'],
        mutation_eta=CONFIG['mutation_eta']
    )
    evox_time = time.time() - start_time
    
    # Evaluate
    y_pred_evox = clf_evox.predict(X_test)
    evox_accuracy = accuracy_score(y_test, y_pred_evox)
    evox_n_rules = len(clf_evox.rule_base.get_rules())
    
    print(f"\n{'=' * 70}")
    print("EVOX RESULTS:")
    print(f"{'=' * 70}")
    print(f"  Training time: {evox_time:.2f}s")
    print(f"  Test accuracy: {evox_accuracy:.4f}")
    print(f"  Number of rules: {evox_n_rules}")
    print(f"\nClassification Report:")
    print(classification_report(y_test, y_pred_evox, target_names=class_names))
else:
    print("⚠️  EvoX backend not available. Skipping EvoX test.")
    print("   To enable EvoX: pip install evox torch")
    evox_time = None
    evox_accuracy = None
    evox_n_rules = None

### Display Learned Rules (EvoX)

In [ ]:
if 'evox' in available:
    print("Learned Fuzzy Rules (EvoX):")
    print("=" * 70)
    rule_str = clf_evox.rule_base.print_rules(return_rules=True)
    print(rule_str)
else:
    print("EvoX not available - skipping rules display")

## Performance Comparison

In [ ]:
# Create comparison table
comparison_data = {
    'Backend': ['PyMoo'],
    'Time (s)': [pymoo_time],
    'Accuracy': [pymoo_accuracy],
    'Rules': [pymoo_n_rules]
}

if 'evox' in available and evox_time is not None:
    comparison_data['Backend'].append('EvoX')
    comparison_data['Time (s)'].append(evox_time)
    comparison_data['Accuracy'].append(evox_accuracy)
    comparison_data['Rules'].append(evox_n_rules)

df_comparison = pd.DataFrame(comparison_data)
print("\n" + df_comparison.to_string(index=False))

# Calculate speedup if both backends are available
if 'evox' in available and evox_time is not None:
    speedup = pymoo_time / evox_time
    print(f"\n📊 Speedup: EvoX is {speedup:.2f}x {'faster' if speedup > 1 else 'slower'} than PyMoo")

## Visualize Comparison

In [ ]:
if MATPLOTLIB_AVAILABLE and 'evox' in available and evox_time is not None:
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Training time comparison
    axes[0].bar(comparison_data['Backend'], comparison_data['Time (s)'], 
                color=['#3498db', '#e74c3c'])
    axes[0].set_ylabel('Time (seconds)')
    axes[0].set_title('Training Time Comparison')
    axes[0].grid(axis='y', alpha=0.3)
    
    # Accuracy comparison
    axes[1].bar(comparison_data['Backend'], comparison_data['Accuracy'], 
                color=['#3498db', '#e74c3c'])
    axes[1].set_ylabel('Accuracy')
    axes[1].set_title('Test Accuracy Comparison')
    axes[1].set_ylim([0, 1.1])
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    print("\n📊 Comparison chart displayed")
elif not MATPLOTLIB_AVAILABLE:
    print("Matplotlib not available - skipping visualization")
else:
    print("EvoX not available - skipping visualization")

## Test with Larger Synthetic Dataset (Optional)

This cell tests the backends with a larger dataset to better demonstrate performance differences.

In [ ]:
# Set to True to run this test (may take several minutes)
RUN_LARGE_DATASET_TEST = False

if RUN_LARGE_DATASET_TEST:
    print("Testing with larger synthetic dataset...\n")
    
    # Create a larger synthetic dataset
    X_large, y_large = make_classification(
        n_samples=CONFIG['large_dataset_samples'],
        n_features=CONFIG['large_dataset_features'],
        n_informative=max(2, CONFIG['large_dataset_features'] - 2),
        n_redundant=min(2, CONFIG['large_dataset_features'] // 4),
        n_classes=3,
        n_clusters_per_class=2,
        random_state=CONFIG['random_state']
    )
    
    X_train_large, X_test_large, y_train_large, y_test_large = train_test_split(
        X_large, y_large, test_size=CONFIG['test_size'], 
        random_state=CONFIG['random_state'], stratify=y_large
    )
    
    print(f"Large dataset:")
    print(f"  Training samples: {len(X_train_large)}")
    print(f"  Test samples: {len(X_test_large)}")
    print(f"  Features: {X_large.shape[1]}")
    
    results_large = {}
    fv_partitions_large = ex_fuzzy.utils.construct_partitions(
        X_train_large, n_partitions=CONFIG['n_linguistic_variables']
    )
    
    # Test PyMoo
    print(f"\nTesting PyMoo on large dataset...")
    clf_pymoo_large = GA_RB.BaseFuzzyRulesClassifier(
        nRules=CONFIG['nRules'], nAnts=CONFIG['nAnts'], 
        n_linguistic_variables=CONFIG['n_linguistic_variables'], 
        verbose=True, backend='pymoo', linguistic_variables=fv_partitions_large
    )
    start = time.time()
    clf_pymoo_large.fit(X_train_large, y_train_large, 
                        n_gen=CONFIG['n_gen'], pop_size=CONFIG['pop_size'])
    results_large['pymoo_time'] = time.time() - start
    results_large['pymoo_acc'] = accuracy_score(
        y_test_large, clf_pymoo_large.predict(X_test_large)
    )
    print(f"PyMoo - Time: {results_large['pymoo_time']:.2f}s, "
          f"Accuracy: {results_large['pymoo_acc']:.4f}")
    
    # Test EvoX if available
    if 'evox' in available:
        print(f"\nTesting EvoX on large dataset...")
        clf_evox_large = GA_RB.BaseFuzzyRulesClassifier(
            nRules=CONFIG['nRules'], nAnts=CONFIG['nAnts'], 
            n_linguistic_variables=CONFIG['n_linguistic_variables'], 
            verbose=True, backend='evox', linguistic_variables=fv_partitions_large
        )
        start = time.time()
        clf_evox_large.fit(X_train_large, y_train_large, 
                          n_gen=CONFIG['n_gen'], pop_size=CONFIG['pop_size'], 
                          sbx_eta=CONFIG['sbx_eta'], mutation_eta=CONFIG['mutation_eta'])
        results_large['evox_time'] = time.time() - start
        results_large['evox_acc'] = accuracy_score(
            y_test_large, clf_evox_large.predict(X_test_large)
        )
        print(f"EvoX - Time: {results_large['evox_time']:.2f}s, "
              f"Accuracy: {results_large['evox_acc']:.4f}")
        
        speedup_large = results_large['pymoo_time'] / results_large['evox_time']
        print(f"\n📊 Large dataset speedup: {speedup_large:.2f}x")
    else:
        print("\n⚠️  EvoX not available for large dataset test")
else:
    print("Large dataset test skipped (set RUN_LARGE_DATASET_TEST = True to run)")

## Summary

### Key Takeaways:

1. ✅ **PyMoo backend** works exactly as before - Full backward compatibility maintained
2. ✅ **EvoX backend** provides GPU acceleration - Potentially faster training on larger datasets
3. ✅ **Easy to switch** backends - Just change the `backend` parameter
4. ✅ **Default behavior unchanged** - Existing code continues to work

### When to use each backend:

- **PyMoo**: Small datasets, CPU-only environments, checkpoint support needed
- **EvoX**: Large datasets, GPU available, need faster training

### Installation:

```bash
# Basic (pymoo only)
pip install ex-fuzzy

# With EvoX support (now uses PyTorch)
pip install evox torch
```

### Additional Resources:

- [ex-fuzzy Documentation](https://github.com/Fuminides/ex-fuzzy)
- [EvoX Library](https://github.com/EMI-Group/evox)
- [PyMoo Library](https://pymoo.org/)